# 추천 모델이란?

matrix factorization or collaborative filtering이란?\
: item들에 대한 user들의 rating 정보를 바탕으로 유사도를 계산하여 아이템을 추천하는 것.\
(유사 그룹을 찾아서 아이템을 추천: user-based CF / 유사 아이템을 찾아서 추천: item-based CF)

# 모델링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from datetime import datetime
from datetime import timedelta

# Windows
mpl.rc("font", family='Malgun Gothic')

warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
df = pd.read_parquet("C://Users//JEON_SANGEON//codestates//data//last.parquet")
df

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658866,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1005008,2053013555631882655,smartphone,xiaomi,94.46,557202408,2169c430-9acc-4db8-8061-ac70ff0c66ea,1.5
7658867,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1003936,2053013555631882655,smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,1.5
7658868,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,1004870,2053013555631882655,smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212,1.5
7658869,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2702331,2053013563911439225,kitchen,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,1.5


In [3]:
df = df.rename(columns={'event_type_y': 'rating'})

참조 : 
https://github.com/journey101/Recommendation_CF_Hybrid_Modeling_with_Python_Surprise/blob/main/Recommendation_CF_Hybrid_modeling_for_Fashion_update(210621).ipynb

In [4]:
item_ = df.groupby(by='product_id').count()
item_

,event_time,month,day,day_name,hour,event_type_x,category_id,category_code,brand,price,user_id,user_session,rating
product_id,,,,,,,,,,,,,
1000978,873,873,873,873,873,873,873,873,873,873,873,873,873
1001588,936,936,936,936,936,936,936,936,936,936,936,936,936
1001606,8,8,8,8,8,8,8,8,8,8,8,8,8
1002042,71,71,71,71,71,71,71,71,71,71,71,71,71
1002062,452,452,452,452,452,452,452,452,452,452,452,452,452
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46800030,1859,1859,1859,1859,1859,1859,1859,1859,1859,1859,1859,1859,1859
46800031,356,356,356,356,356,356,356,356,356,356,356,356,356
46800034,2,2,2,2,2,2,2,2,2,2,2,2,2


In [5]:
user_ = df.groupby(by='user_id').count()
user_

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_session,rating
user_id,,,,,,,,,,,,,
226242984,1,1,1,1,1,1,1,1,1,1,1,1,1
231943435,1,1,1,1,1,1,1,1,1,1,1,1,1
239475080,2,2,2,2,2,2,2,2,2,2,2,2,2
241784978,1,1,1,1,1,1,1,1,1,1,1,1,1
244951053,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
566279874,1,1,1,1,1,1,1,1,1,1,1,1,1
566280187,1,1,1,1,1,1,1,1,1,1,1,1,1
566280291,3,3,3,3,3,3,3,3,3,3,3,3,3


In [6]:
user_.sort_values(by="product_id", ascending=False)

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_session,rating
user_id,,,,,,,,,,,,,
512505687,1407,1407,1407,1407,1407,1407,1407,1407,1407,1407,1407,1407,1407
512365995,1364,1364,1364,1364,1364,1364,1364,1364,1364,1364,1364,1364,1364
513021392,1159,1159,1159,1159,1159,1159,1159,1159,1159,1159,1159,1159,1159
563459593,908,908,908,908,908,908,908,908,908,908,908,908,908
535925182,783,783,783,783,783,783,783,783,783,783,783,783,783
...,...,...,...,...,...,...,...,...,...,...,...,...,...
557393973,1,1,1,1,1,1,1,1,1,1,1,1,1
557393971,1,1,1,1,1,1,1,1,1,1,1,1,1
557393970,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
df.groupby(by='category_code').count()

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,brand,price,user_id,user_session,rating
category_code,,,,,,,,,,,,,
kitchen,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187,1939187
notebook,465265,465265,465265,465265,465265,465265,465265,465265,465265,465265,465265,465265,465265
smartphone,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299,4714299
video,540120,540120,540120,540120,540120,540120,540120,540120,540120,540120,540120,540120,540120


product_id별 user_id : 1:n관계 \
user_id별 product_id : 1:n관계 \
category_code 별 product_id : 1:n관계 

추천 시스템에 적용하기에 적합한 데이터로 판단됨. (유사 유저그룹 판단 후 아직 경험하지 않은 아이템을 추천해야 하기 때문)

In [4]:
# rating을 value로 놓고 진행함.

# product_id 관심도를 활용하여 value_average 추가 후 merge

mean_df = df[['product_id','rating']].groupby(by='product_id').mean()
mean_df = mean_df.reset_index()
mean_df = mean_df.rename(columns={'rating': 'rating_average'})
mean_df

,product_id,rating_average
0,1000978,2.344559
1,1001588,2.405342
2,1001606,1.500000
3,1002042,3.005634
4,1002062,2.521239
...,...,...
15996,46800030,1.605648
15997,46800031,1.667697
15998,46800034,1.500000
15999,46800035,1.500000


In [18]:
mean_df['rating_average'].value_counts()

1.500000    3686
2.300000     108
2.033333     107
3.100000     107
5.000000      74
            ... 
1.865854       1
2.106667       1
1.800840       1
2.287652       1
1.667697       1
Name: rating_average, Length: 8187, dtype: int64

In [5]:
df_r_m = pd.merge(df, mean_df, how='left', on='product_id')
df_r_m['user_id'] = df_r_m['user_id'].astype(int)
df_r_m['product_id'] = df_r_m['product_id'].astype(int)
# df_r_merge.dtypes

df_r_m.head()

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,rating,rating_average
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0,2.960048
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5,3.108415
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5,2.373963
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1,2.960048
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5,1.655698


In [11]:
df_r_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7658871 entries, 0 to 7658870
Data columns (total 15 columns):
 #   Column          Dtype                     
---  ------          -----                     
 0   event_time      datetime64[ns, Asia/Dubai]
 1   month           int64                     
 2   day             int64                     
 3   day_name        object                    
 4   hour            int64                     
 5   event_type_x    object                    
 6   product_id      int32                     
 7   category_id     int64                     
 8   category_code   object                    
 9   brand           object                    
 10  price           float64                   
 11  user_id         int32                     
 12  user_session    object                    
 13  rating          float64                   
 14  rating_average  float64                   
dtypes: datetime64[ns, Asia/Dubai](1), float64(3), int32(2), int64(4), 

In [6]:
df_r_m['rating'].value_counts()

1.5    4688667
5.0    1620976
3.4     896815
3.1     452259
1.9         86
1.6         35
3.5         33
Name: rating, dtype: int64

In [7]:
df_r_m.columns

Index(['event_time', 'month', 'day', 'day_name', 'hour', 'event_type_x',
       'product_id', 'category_id', 'category_code', 'brand', 'price',
       'user_id', 'user_session', 'rating', 'rating_average'],
      dtype='object')

# 베이스라인 모형

https://datascienceschool.net/03%20machine%20learning/07.01%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C.html

## 모형 사용법

1. 데이터세트의 split, folds 메소드를 사용하여 K-Fold 트레이닝 데이터셋과 테스트 데이터셋을 만든다.
2. 모형 알고리즘 객체를 생성한다.
3. 모형 알고리즘 객체의 train 메서드와 트레이닝 데이터셋으로 모수를 추정한 후, test 메서드로 테스트 데이터셋에 대한 예측을 실시한다.
4. accuracy 서브패키지의 성능평가 함수를 사용하여 예측 성능을 계산한다.

이 과정은 evaluate 명령으로 단축할 수도 있다.

surprise 패키지는 베이스라인 모형을 위한 BaselineOnly 클래스를 제공한다.

In [12]:
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic
from surprise import accuracy
from surprise import dump
from surprise.model_selection import KFold
from surprise import NormalPredictor
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import train_test_split

In [7]:
# To load a dataset from a pandas dataframe, you will need the 'load_from_df()' method.

# 판다스 데이터 프레임에서 데이터 세트를 로드하려면 아래와 같은 메소드를 먼저 실행해주어야 합니다.

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_r_m[['user_id', 'product_id', 'rating']], reader)

In [12]:
# We can now use this dataset as we please, e.g. calling cross_validate
# NormalPredictor - 정규라고 가정되는 훈련 세트의 분포를 기반으로 무작위 등급을 예측하는 알고리즘입니다.
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.87180346, 1.87232734]),
 'test_mae': array([1.51225909, 1.51237597]),
 'fit_time': (7.483814001083374, 10.768038034439087),
 'test_time': (35.77755045890808, 39.11675429344177)}

Use cross-validation iterators

In [13]:
# define a cross-validation iterator
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.4449
RMSE: 0.4469
RMSE: 0.4478


In [8]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0409603465618413
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [9]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [10]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

In [11]:
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,1.296432,1.296858,1.296221,1.296504,0.000265,8,1.016657,1.016584,1.016146,1.016463,0.000226,7,73.522929,1.526772,30.322342,1.766894,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,1.287164,1.287544,1.286940,1.287216,0.000250,7,1.032519,1.032507,1.032031,1.032352,0.000227,8,70.064150,0.032207,29.516647,1.866763,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6
2,1.170801,1.170927,1.170338,1.170689,0.000253,4,0.875137,0.875126,0.874565,0.874943,0.000267,3,68.074668,2.259669,28.517122,2.937361,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}",5,0.005,0.4
3,1.165248,1.165324,1.164794,1.165122,0.000234,3,0.896652,0.896657,0.896037,0.896449,0.000291,4,66.161833,1.443637,28.441969,0.355339,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}",5,0.005,0.6
4,1.184192,1.184806,1.184070,1.184356,0.000322,6,0.914931,0.915047,0.914522,0.914834,0.000225,5,137.913753,0.074761,30.524176,1.699283,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}",10,0.002,0.4
5,1.182127,1.182656,1.181964,1.182249,0.000295,5,0.937196,0.937323,0.936789,0.937103,0.000228,6,137.719418,0.498499,31.264984,0.965854,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}",10,0.002,0.6
6,1.040996,1.041322,1.040563,1.040960,0.000311,1,0.770345,0.770331,0.769773,0.770150,0.000266,1,136.928326,0.661831,29.923177,1.633176,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}",10,0.005,0.4
7,1.048537,1.048684,1.048029,1.048417,0.000281,2,0.795392,0.795426,0.794829,0.795216,0.000274,2,136.648525,0.596094,29.188898,1.698510,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}",10,0.005,0.6


# 모델 분석방법

# Model-based Collaborative Filtering

- Clustering-based Algorithm
- Matrix Factorization Algorithm
- Deep Learning

세가지로 분류되는데 그 중 Clustering-based Algorithm에 surprise KNN을 활용예정


surprise 모듈 임포트

Surprise
- 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트에 적용이 가능한 모듈

train, test set

In [13]:
# surprise 모듈의 훈련,테스트 모듈 나누는것과 비슷한 방법으로 데이터 분리 진행

# train, test split

# 데이터 랜덤하게 섞기
df_r_m = df.sample(frac=1)

# 7:3 비율로 나누기
cutoff = int(.7 * len(df_r_m))
train_r = df_r_m[:cutoff]
test_r = df_r_m[cutoff:]


train_r = train_r[['user_id', 'product_id', 'rating', 'category_code', 'brand']]
test_r = test_r[['user_id', 'product_id', 'rating', 'category_code', 'brand']]
print("train_size:", len(train_r), "test_size:", len(test_r))

train_r.head()

train_size: 5361209 test_size: 2297662


,user_id,product_id,rating,category_code,brand
6471075,534275871,1004708,3.1,smartphone,huawei
1560517,544436647,1004659,1.5,smartphone,samsung
2827098,535886556,1004250,5.0,smartphone,apple
3554791,522235312,2800636,1.5,kitchen,indesit
608023,525624569,3601255,1.5,kitchen,no_brand


In [14]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(1, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'product_id', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'product_id', 'rating']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    
    return trainset, testset

trainset, testset = convert_traintest_dataframe_forsurprise(train_r, test_r)

Surprise 추천알고리즘 클래스

KNNBaseline - 기준 등급 을 고려한 기본 협업 필터링 알고리즘 입니다.

SVD - Netflix Prize 기간 동안 Simon Funk 가 대중화한 유명한 SVD 알고리즘입니다. 기준선을 사용하지 않는 경우 이는 확률적 행렬 분해[salakhutdinov2008a]와 동일합니다(아래 참고 참조).

SVDpp - SVD++ 알고리즘은 SVD 암시적 등급 고려의 확장 입니다.


https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

참고 : 
https://github.com/prakruti-joshi/Movie-Recommendation-System/blob/master/Code/cold_start_analysis.ipynb

## (1) Knn-baseline 모델

- 기준 등급 을 고려한 기본 협업 필터링 알고리즘 입니다.

surprise.prediction_algorithms.knns.KNNBaseline( k=40 , min_k=1 , sim_options={} , bsl_options={} , verbose=True , **kwargs )

product_id 기반 모델

In [15]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# content-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html


sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)

file_name = 'KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_predictions_i)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions_i)
accuracy.mae(knnbaseline_predictions_i)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.4316
MAE:  0.2192
Done!


user_id 기반 모델 (메모리 초과로 진행불가)

In [ ]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# user-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html

# sim_options = {'name': 'cosine',
#                'user_based': True  # user-based similarity
#                }
# knnbaseline_algo_u = KNNBaseline(sim_options=sim_options)

# knnbaseline_algo_u.fit(trainset)
# knnbaseline_predictions_u = knnbaseline_algo_u.test(testset)

# file_name = 'KnnBaseline_model_u'
# dump.dump(file_name, algo=knnbaseline_predictions_u)
# # _, loaded_algo = dump.load(file_name)

# accuracy.rmse(knnbaseline_predictions_u)
# accuracy.mae(knnbaseline_predictions_u)
# print("Done!")

In [16]:
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)
knnbaseline_predictions_i[:3]

[Prediction(uid=564085761, iid=1004246, r_ui=1.5, est=1.7705682770890259, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=558992205, iid=1003317, r_ui=1.5, est=2.3920117556500182, details={'was_impossible': False}),
 Prediction(uid=552065728, iid=3600145, r_ui=1.5, est=1.5, details={'actual_k': 1, 'was_impossible': False})]

In [17]:
# uid, iid, r_ui(실제 평점) ,est(예측평점) 추출

[(pred.uid, pred.iid, pred.est) for pred in knnbaseline_predictions_i[:3]]

[(564085761, 1004246, 1.7705682770890259),
 (558992205, 1003317, 2.3920117556500182),
 (552065728, 3600145, 1.5)]

In [18]:
# uid = 564085761 에 대한 유사도 높은 예측 iid, est 조회. 
# 참고: https://hmkim312.github.io/posts/%EC%84%9C%ED%94%84%EB%9D%BC%EC%9D%B4%EC%A6%88(Surprise)%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommendations)/

pred_df = pd.DataFrame(knnbaseline_predictions_i)
pred_df[(pred_df['uid']==564085761)]

,uid,iid,r_ui,est,details
0,564085761,1004246,1.5,1.770568,"{'actual_k': 6, 'was_impossible': False}"
819243,564085761,1004258,1.5,1.908919,"{'actual_k': 9, 'was_impossible': False}"
1553182,564085761,1003316,1.5,1.840554,"{'actual_k': 9, 'was_impossible': False}"
1952502,564085761,1004252,1.5,1.566236,"{'actual_k': 6, 'was_impossible': False}"
2187687,564085761,31501004,1.5,1.254045,"{'actual_k': 4, 'was_impossible': False}"


## (2) svd, svdpp (CF) 모델

In [19]:
 ## svd 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms

svd_algo = SVD()

svd_algo.fit(trainset)
svd_predictions = svd_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svd_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svd_predictions)
accuracy.mae(svd_predictions)
print("Done!")

RMSE: 0.4463
MAE:  0.3396
Done!


In [20]:
# matrix factorization/ collaborative filtering(CF)
 ## svd++ model : taking into account implicit ratings 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp

svdpp_algo = SVDpp()

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svdpp_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 2.8310
MAE:  2.4428
Done!


## (3) gridsearchCV 모델 오래걸려서 중단

In [ ]:
# from surprise import SVD
# from surprise import Dataset
# from surprise import Reader
# from surprise.model_selection import GridSearchCV

# param_grid = {'n_epochs': [5, 20], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
#               # lr_all – The learning rate for all parameters. Default is 0.005.
#               # reg_all – The regularization term for all parameters. Default is 0.02.
              
# gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5) 

# reader = Reader()

# # The columns must correspond to user id, product_id and ratings (in that order).
# train_data = Dataset.load_from_df(train_r[['user_id', 'product_id', 'rating']], reader)

# gs.fit(train_data)

# # best RMSE score
# print("gs.best_score(rmse): ", gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print("gs.best_params: ", gs.best_params['rmse'])

In [ ]:
# from surprise import accuracy

# # best model fit 
# # We can now use the algorithm that yields the best rmse:
# GS_SVDpp_best = gs.best_estimator['rmse']

# # retrain on the whole set train
# trainset = train_data.build_full_trainset()
# GS_SVDpp_best.fit(trainset)

# # Compute biased accuracy on train (validation set)
# predictions = GS_SVDpp_best.test(trainset.build_testset())
# print('validation test,', end='\n')
# accuracy.rmse(predictions)
# accuracy.mae(predictions)
# # 참고:https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation
# ## 참고: https://surprise.readthedocs.io/en/stable/getting_started.html#train-on-whole-trainset

In [ ]:
# # Compute biased accuracy on train (validation set)
# predictions = GS_SVDpp_best.test(testset)
# print('testset accuracy,', end='\n')
# accuracy.rmse(predictions)
# accuracy.mae(predictions)

### 1-2) evaluation (knn/svd/svdpp 비교)

- knnbaseline (item_based)
- SVD (CF)
- SVDpp (CF)

prediction, rmse, mae, precision, recall, f-measure, ndcg_score 계산값

In [22]:
# predictions 
knn_i_pred = knnbaseline_algo_i.test(testset)
svd_pred = svd_algo.test(testset)
svdpp_pred = svdpp_algo.test(testset)

In [23]:
# test_rmse, test_mae
knn_i_rmse = accuracy.rmse(knn_i_pred)
knn_i_mae = accuracy.mae(knn_i_pred)
svd_rmse = accuracy.rmse(svd_pred)
svd_mae = accuracy.mae(svd_pred)
svdpp_rmse = accuracy.rmse(svdpp_pred)
svdpp_mae = accuracy.mae(svdpp_pred)

RMSE: 0.4316
MAE:  0.2192
RMSE: 0.4463
MAE:  0.3396
RMSE: 2.8310
MAE:  2.4428


In [24]:
# recall & precision @ k function
# 참고: https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=recall#how-to-compute-precision-k-and-recall-k

from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [25]:
# knn_i 모델 precision&recall
knn_i_prec, knn_i_re = precision_recall_at_k(knn_i_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
knn_i_prec_avg = sum(prec for prec in knn_i_prec.values()) / len(knn_i_prec)
print(knn_i_prec_avg)
knn_i_re_avg = sum(rec for rec in knn_i_re.values()) / len(knn_i_re)
print(knn_i_re_avg)

0.12248232875149963
0.10366478709755178


In [26]:

# svd precision&recall
svd_prec, svd_re = precision_recall_at_k(svd_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svd_prec_avg = sum(prec for prec in svd_prec.values()) / len(svd_prec)
print(svd_prec_avg)
svd_re_avg = sum(rec for rec in svd_re.values()) / len(svd_re)
print(svd_re_avg)

0.11253895689540903
0.08951768021956362


In [27]:
# svdpp precision&recall
svdpp_prec, svdpp_re = precision_recall_at_k(svdpp_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svdpp_prec_avg = sum(prec for prec in svdpp_prec.values()) / len(svdpp_prec)
print(svdpp_prec_avg)
svdpp_re_avg = sum(rec for rec in svdpp_re.values()) / len(svdpp_re)
print(svdpp_re_avg)

0.12492971574953085
0.10966208195674963


In [28]:
# model별 test prediction  데이터 
# row: Knnbaseline(item based), Knnbaseline(user based), SVD(CF), SVDpp(CF), gridsearch(SVDpp)
# columns: test RMSE, test MAE, test_time, Precision, recall
# hybrid 모델 수치는 SVDpp(CF)모델과 Knnbaseline(item-based)의 평균값. 
test_prediction = pd.DataFrame({"Algorithm": ["KnnBaseline(item-based)" ,  
                                              "SVD(CF)",
                                              "SVDpp(CF)"

                                              ]
                                ,"test_rmse": [knn_i_rmse , svd_rmse , svdpp_rmse]
                                ,"test_mae":[knn_i_mae , svd_mae, svdpp_mae]
                                ,"Precision":[knn_i_prec_avg ,svd_prec_avg ,svdpp_prec_avg]
                                ,"recall":[knn_i_re_avg ,svd_re_avg ,svdpp_re_avg]
                                })

test_prediction

,Algorithm,test_rmse,test_mae,Precision,recall
0,KnnBaseline(item-based),0.431577,0.219185,0.122482,0.103665
1,SVD(CF),0.446251,0.339567,0.112539,0.089518
2,SVDpp(CF),2.830991,2.442792,0.124930,0.109662


모델을 활용한 추천시스템 제작준비중

# (수정) 아이템 기반 추천시스템 제작

- 데이터 가공하여 파케이 파일 제작하여 위에서 진행중
- view, cart, purchase를 점수화하여 평점으로 만드는 작업 추가함.
- 코사인 유사도 활용
- 참고는 아래 블로그 내용들 참고하여 이것저것 활용함.



위의 서프라이즈 모듈 활용은 아직 학습중!!

- Memory-based Collaborative Filtering으로 item-based collaborative Filtering
- 평가는 MSE 활용함.

블로그 참조 :

https://kutar37.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81Collaborative-Filtering-%EC%B6%94%EC%B2%9C-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-1

https://western-sky.tistory.com/58

https://velog.io/@jumpx2/python-%EC%B5%9C%EA%B7%BC%EC%A0%91-%EC%9D%B4%EC%9B%83-%ED%98%91%EC%97%85-%ED%95%84%ED%84%B0%EB%A7%81

In [2]:
# 금,토,일 한정
# df = pd.read_parquet("C://Users//JEON_SANGEON//codestates//data//sample.parquet")
# df

# 모든 요일 카테고리 분류전
df = pd.read_parquet("C://Users//JEON_SANGEON//codestates//data//cp2_data.parquet")
df

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42448759,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [3]:
df = df.loc[(df['price']!=0)]

In [4]:
# df copy < 선호도 기준을 view 기준으로 비율화하면 좋을듯 했으나 해당 내용으로 봐도될지 근거가 부족하여 일단 통합하여 진행

# df_v = df.loc[df['event_type']=='view']
df_v = df.copy()

In [5]:
# 필요한 컬럼만 사용

df_v = df_v[['user_id','category_code','event_type']]
df_v

,user_id,category_code,event_type
0,541312140,no_cat,view
1,554748717,appliances.environment.water_heater,view
2,519107250,furniture.living_room.sofa,view
3,550050854,computers.notebook,view
4,535871217,electronics.smartphone,view
...,...,...,...
42448759,537931532,electronics.camera.video,view
42448760,527322328,appliances.kitchen.multibaker,view
42448761,566280422,auto.accessories.player,view
42448762,513118352,electronics.clocks,view


In [6]:
df_v = df_v.drop_duplicates()
df_v.event_type.value_counts()

view        6989144
purchase     433130
cart         381975
Name: event_type, dtype: int64

In [7]:
event_type_strength = {
   'view': 1.5,
   'cart': 1.9, 
   'purchase': 1.6
}

df_v['event_type'] = df_v['event_type'].apply(lambda x: event_type_strength[x])

# 참고
# https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101/notebook
# Data munging 부분에서 event_type으로 점수 매김.

In [8]:
df_v

,user_id,category_code,event_type
0,541312140,no_cat,1.5
1,554748717,appliances.environment.water_heater,1.5
2,519107250,furniture.living_room.sofa,1.5
3,550050854,computers.notebook,1.5
4,535871217,electronics.smartphone,1.5
...,...,...,...
42448711,553074032,computers.peripherals.mouse,1.5
42448723,566280860,missing,1.5
42448744,566031393,electronics.smartphone,1.5
42448755,544578298,no_cat,1.5


In [9]:
# 선호도 확인을 위한 그룹화 진행

df_v_g = df_v.groupby(['user_id','category_code'])['event_type'].sum().reset_index()
df_v_g

,user_id,category_code,event_type
0,33869381,kids.carriage,1.5
1,183503497,missing,1.5
2,184265397,furniture.living_room.chair,1.5
3,184265397,no_cat,1.5
4,195082191,electronics.audio.headphone,1.5
...,...,...,...
6989477,566280663,electronics.smartphone,1.5
6989478,566280676,furniture.bedroom.bed,1.5
6989479,566280697,electronics.camera.video,1.5
6989480,566280780,furniture.bedroom.set,1.5


In [10]:
df_v_m = pd.merge(left=df, right=df_v_g,how='left',\
                    on=['user_id','category_code'],sort=False)

df_v_m

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,3.1
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,1.5
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,1.5
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,1.5
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42380086,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,1.5
42380087,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,1.5
42380088,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,1.5
42380089,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,1.5


In [20]:
df_day = df_v_m.loc[(df_v_m['day_name']=='Friday')|
                (df_v_m['day_name']=='Saturday')|
                (df_v_m['day_name']=='Sunday')
                ]

In [21]:
df_day['category_code'] = df_day["category_code"].str.split(".").str[1]

In [23]:
df_cat = df_day.loc[(df_day['category_code']=='smartphone')|
                    (df_day['category_code']=='video')|
                    (df_day['category_code']=='kitchen')|
                    (df_day['category_code']=='notebook')
                    ]

In [24]:
df_cat=df_cat.reset_index(drop=True)
df_cat

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658866,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1005008,2053013555631882655,smartphone,xiaomi,94.46,557202408,2169c430-9acc-4db8-8061-ac70ff0c66ea,1.5
7658867,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1003936,2053013555631882655,smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,1.5
7658868,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,1004870,2053013555631882655,smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212,1.5
7658869,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2702331,2053013563911439225,kitchen,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,1.5


In [25]:
df_cat.to_parquet('last.parquet', engine="pyarrow", compression="gzip")

In [29]:
ratings_matrix = df_v_g.pivot_table('event_type', 'user_id', 'category_code')
# pivot_table(value, index, columns) 
# 사용자별 평점 
ratings_matrix.fillna(0, inplace=True)
#NaN = 0 으로 채우기
ratings_matrix_T = ratings_matrix.T
# 영화별 사용자 평점 
# index <-> columns 위치를 바꾸기
# 아이템기반 협업 필터링을 하기위해 사용자-아이템 행렬을 아이템-사용자 행렬을 사용
ratings_matrix.head(3)

category_code,accessories.bag,accessories.jewelry,accessories.umbrella,accessories.wallet,apparel.belt,apparel.coat,apparel.costume,apparel.dress,apparel.jacket,apparel.jeans,...,kids.toys,medicine.tools.tonometer,missing,no_cat,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
user_id,,,,,,,,,,,,,,,,,,,,,
33869381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183503497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184265397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
ratings_matrix

category_code,accessories.bag,accessories.jewelry,accessories.umbrella,accessories.wallet,apparel.belt,apparel.coat,apparel.costume,apparel.dress,apparel.jacket,apparel.jeans,...,kids.toys,medicine.tools.tonometer,missing,no_cat,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
user_id,,,,,,,,,,,,,,,,,,,,,
33869381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183503497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184265397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
195082191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200673532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566280663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566280676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566280697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
ratings_matrix.T.head(3)

user_id,33869381,183503497,184265397,195082191,200673532,205053188,208669541,209714031,210507313,216064734,...,566280392,566280399,566280416,566280422,566280536,566280663,566280676,566280697,566280780,566280860
category_code,,,,,,,,,,,,,,,,,,,,,
accessories.bag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accessories.jewelry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accessories.umbrella,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

sklearn_cosine = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

item_similarity = pd.DataFrame(sklearn_cosine, index=ratings_matrix_T.index, columns = ratings_matrix_T.index)

item_similarity.head()

category_code,accessories.bag,accessories.jewelry,accessories.umbrella,accessories.wallet,apparel.belt,apparel.coat,apparel.costume,apparel.dress,apparel.jacket,apparel.jeans,...,kids.toys,medicine.tools.tonometer,missing,no_cat,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
category_code,,,,,,,,,,,,,,,,,,,,,
accessories.bag,1.000000,0.082012,0.075985,0.147740,0.063285,0.055334,0.068168,0.049949,0.006485,0.059070,...,0.043964,0.015177,0.137226,0.151129,0.048159,0.014424,0.020793,0.019919,0.028313,0.016307
accessories.jewelry,0.082012,1.000000,0.034625,0.074694,0.029582,0.108369,0.090134,0.076606,0.011055,0.071369,...,0.062023,0.021762,0.185127,0.173159,0.032194,0.005416,0.004854,0.006230,0.024374,0.010078
accessories.umbrella,0.075985,0.034625,1.000000,0.097595,0.066315,0.025449,0.023970,0.023702,0.001146,0.025002,...,0.016855,0.010320,0.036514,0.036479,0.010142,0.003339,0.001148,0.003875,0.007466,0.004331
accessories.wallet,0.147740,0.074694,0.097595,1.000000,0.131061,0.041373,0.045599,0.038209,0.005651,0.040034,...,0.025647,0.014353,0.067078,0.068609,0.014944,0.002994,0.003732,0.007672,0.012530,0.005987
apparel.belt,0.063285,0.029582,0.066315,0.131061,1.000000,0.019664,0.026675,0.016758,0.001472,0.024587,...,0.010026,0.003615,0.030184,0.029934,0.009324,0.007290,0.003686,0.004147,0.005901,0.001854


In [32]:
item_similarity[df['category_code'][300]].sort_values(ascending=False)[:5]
# df['category_code'][300] = 'electronics.smartphone'

# 평가 점수와 유사한 카테고리를 추천 

category_code
electronics.smartphone                      1.000000
no_cat                                      0.187133
electronics.clocks                          0.169941
electronics.audio.headphone                 0.159087
electronics.smartphone.phone_accessories    0.151183
Name: electronics.smartphone, dtype: float64

In [39]:
# 스마트폰과 유사도가 높은 카테고리 5개만 추출하기
item_similarity['electronics.smartphone'].sort_values(ascending=False)[:6]

category_code
electronics.smartphone                      1.000000
no_cat                                      0.187133
electronics.clocks                          0.169941
electronics.audio.headphone                 0.159087
electronics.smartphone.phone_accessories    0.151183
missing                                     0.148628
Name: electronics.smartphone, dtype: float64

In [40]:
# appliances.kitchen.dishwasher과 유사도가 높은 카테고리 5개만 추출하기
item_similarity['appliances.kitchen.dishwasher'].sort_values(ascending=False)[:6]

category_code
appliances.kitchen.dishwasher       1.000000
appliances.kitchen.refrigerators    0.155388
no_cat                              0.126449
appliances.environment.vacuum       0.117942
missing                             0.115597
appliances.kitchen.cooker           0.115447
Name: appliances.kitchen.dishwasher, dtype: float64

### 개인화된 카테고리 추천 시스템

위의 추천방식은 단순 카테고리간의 유사도만으로 추천을 진행했습니다.

이번엔 개인의 평점이 반영된 추천시스템을 구현하겠습니다.

In [34]:
# def predict_rating(ratings_array, item_similarity):
#     sum_sr = ratings_array @ item_similarity
#             # u x i , item_similarity = ixi
#         # @ 연산자는 np.matmul, np.dot 행렬 곱을 해줌
#     sum_abs = np.array([np.abs(item_similarity).sum(axis=1)])
    
#     ratings_pred = sum_sr / sum_abs
#     return ratings_pred

# rating_pred = predict_rating(ratings_matrix.values, item_similarity.values)

# rating_pred_matrix = pd.DataFrame(data=rating_pred, index=ratings_matrix.index, columns = ratings_matrix.columns)

# rating_pred_matrix

category_code,accessories.bag,accessories.jewelry,accessories.umbrella,accessories.wallet,apparel.belt,apparel.coat,apparel.costume,apparel.dress,apparel.jacket,apparel.jeans,...,kids.toys,medicine.tools.tonometer,missing,no_cat,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
user_id,,,,,,,,,,,,,,,,,,,,,
33869381,0.003139,0.007489,0.003051,0.003702,0.002264,0.006007,0.005021,0.005032,0.001537,0.003493,...,0.016497,0.003326,0.006926,0.006728,0.005790,0.001394,0.001294,0.001809,0.002565,0.001773
183503497,0.020625,0.028378,0.012835,0.017057,0.011714,0.027501,0.022184,0.019857,0.009372,0.020096,...,0.025757,0.013133,0.086397,0.033067,0.022488,0.013236,0.012597,0.012712,0.024229,0.015415
184265397,0.028301,0.031406,0.016357,0.021153,0.015348,0.025817,0.024190,0.021615,0.012376,0.021597,...,0.034904,0.017615,0.041198,0.087316,0.027340,0.014124,0.018411,0.013861,0.026263,0.016414
195082191,0.008988,0.008375,0.004781,0.007127,0.004818,0.004958,0.007036,0.003261,0.001407,0.006577,...,0.006373,0.004570,0.009182,0.012251,0.008303,0.005374,0.005986,0.005652,0.005707,0.004255
200673532,0.004885,0.001732,0.001564,0.002384,0.002138,0.000993,0.002487,0.000386,0.000000,0.002425,...,0.002527,0.001723,0.007576,0.008446,0.005575,0.005620,0.003777,0.002957,0.003869,0.007702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566280663,0.009456,0.017629,0.004517,0.007635,0.004081,0.010059,0.011031,0.006980,0.002739,0.009517,...,0.010111,0.006281,0.012841,0.015269,0.011398,0.005296,0.005862,0.005180,0.007543,0.005716
566280676,0.006830,0.013434,0.005209,0.006734,0.004645,0.011326,0.009038,0.010146,0.005205,0.008375,...,0.011853,0.007334,0.014265,0.012072,0.007023,0.003112,0.002677,0.003792,0.007544,0.003871
566280697,0.003792,0.001620,0.002391,0.002207,0.001175,0.001043,0.002015,0.000855,0.000486,0.001808,...,0.002350,0.001340,0.003123,0.005258,0.004793,0.005751,0.006420,0.002922,0.002614,0.002926


In [49]:
# ratings_arr.dot(item_sim_arr)는 평점 * 맥주 유사도
# ratings_arr는 사용자 u의 아이템 i와 가장 유사도가 높은 Top_N개 아이템에 대한 실제 평점 벡터
# item_sim_arr는 아이템 i와 가장 유사도가 높은 Top_N개 아이템의 유사도 벡터
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# 개인화된 예측 평점 구하기
# 평점 value와 유사도 value만 뽑아서 대입
ratings_pred = predict_rating(ratings_matrix.values, item_similarity.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns = ratings_matrix.columns)

# 개인별로 계산된 예측 평점
ratings_pred_matrix

category_code,accessories.bag,accessories.jewelry,accessories.umbrella,accessories.wallet,apparel.belt,apparel.coat,apparel.costume,apparel.dress,apparel.jacket,apparel.jeans,...,kids.toys,medicine.tools.tonometer,missing,no_cat,sport.bicycle,sport.ski,sport.snowboard,sport.tennis,sport.trainer,stationery.cartrige
user_id,,,,,,,,,,,,,,,,,,,,,
33869381,0.003139,0.007489,0.003051,0.003702,0.002264,0.006007,0.005021,0.005032,0.001537,0.003493,...,0.016497,0.003326,0.006926,0.006728,0.005790,0.001394,0.001294,0.001809,0.002565,0.001773
183503497,0.020625,0.028378,0.012835,0.017057,0.011714,0.027501,0.022184,0.019857,0.009372,0.020096,...,0.025757,0.013133,0.086397,0.033067,0.022488,0.013236,0.012597,0.012712,0.024229,0.015415
184265397,0.028301,0.031406,0.016357,0.021153,0.015348,0.025817,0.024190,0.021615,0.012376,0.021597,...,0.034904,0.017615,0.041198,0.087316,0.027340,0.014124,0.018411,0.013861,0.026263,0.016414
195082191,0.008988,0.008375,0.004781,0.007127,0.004818,0.004958,0.007036,0.003261,0.001407,0.006577,...,0.006373,0.004570,0.009182,0.012251,0.008303,0.005374,0.005986,0.005652,0.005707,0.004255
200673532,0.004885,0.001732,0.001564,0.002384,0.002138,0.000993,0.002487,0.000386,0.000000,0.002425,...,0.002527,0.001723,0.007576,0.008446,0.005575,0.005620,0.003777,0.002957,0.003869,0.007702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566280663,0.009456,0.017629,0.004517,0.007635,0.004081,0.010059,0.011031,0.006980,0.002739,0.009517,...,0.010111,0.006281,0.012841,0.015269,0.011398,0.005296,0.005862,0.005180,0.007543,0.005716
566280676,0.006830,0.013434,0.005209,0.006734,0.004645,0.011326,0.009038,0.010146,0.005205,0.008375,...,0.011853,0.007334,0.014265,0.012072,0.007023,0.003112,0.002677,0.003792,0.007544,0.003871
566280697,0.003792,0.001620,0.002391,0.002207,0.001175,0.001043,0.002015,0.000855,0.000486,0.001808,...,0.002350,0.001340,0.003123,0.005258,0.004793,0.005751,0.006420,0.002922,0.002614,0.002926


In [54]:
def no_watch(ratings_matrix, user):
    user_rating = ratings_matrix.loc[user,:]
    #user_id 별로 평점 정보
    no_watching = user_rating[user_rating==0].index.tolist()
    # 보지않은 카테고리 리스트
    category_list = ratings_matrix.columns.tolist()
    # 모든 카테고리를 리스트 만들기
    no_list = [category for category in category_list if category in no_watching]
    # 안 본 영화 리스트 
    return no_list

def recommend_category_by_userid(pred_df, userId, no_list, top_n=10):
    # 안 본 카테고리 리스트 중 예측 점수가 가장 높은 순서 .series
    recommend_category = pred_df.loc[userId, no_list].sort_values(ascending=False)[:top_n]
    return recommend_category

user_id = 200673532
no_watch = no_watch(ratings_matrix, user_id)

recommend_category = recommend_category_by_userid(ratings_pred_matrix, user_id, no_watch, top_n=10)

recom_categories = pd.DataFrame(data= recommend_category.values, index= recommend_category.index, columns=['pred_score'])
recom_categories

,pred_score
category_code,
computers.components.cpu,0.183994
computers.components.power_supply,0.155773
computers.components.memory,0.155657
computers.components.hdd,0.099120
computers.components.cooler,0.081036
computers.peripherals.monitor,0.080141
computers.peripherals.keyboard,0.060594
computers.peripherals.mouse,0.050879
computers.desktop,0.033797


### 우리가 예측한 평점과 실제 평점간의 차이를 MSE로 계산

In [53]:
from sklearn.metrics import mean_squared_error

# 우리가 예측한 평점과 실제 평점간의 차이를 MSE로 계산
def get_mse(pred, actual):
    # 평점이 있는 실제 카테고리만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  1.408323237099428
